In [1]:
# Import dependencies 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.
#COMPETITION_PATH = Path(r"/kaggle/input/llm-prompt-recovery/")
#INPUT_PATH = Path(r"/kaggle/input")
import keras
import keras_nlp
import random

# ---- HYPER PARAMATER TUNING ----

# Class setup for Gemma
class CFG:
    # don't need to change this unless you want to reset data for the session
    seed = 42
    # don't change
    dataset_path = "/kaggle/input/llm-prompt-recovery"
    # Model Preset - important for specifying the open source model
    preset = "gemma_instruct_2b_en"
    # Max input length for the text data going into model during Fine-Tuning
    sequence_length = 512
    # Max output length for generation from model
    max_len = 512
    # Minimal change when increased
    batch_size = 1 
    # Minimal change when increased 
    epochs = 1 
    
# Log progress
print("end of dependencies")

/kaggle/input/testblendrewrite/rewritten.csv
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/llm-prompt-recovery/sample_submission.csv
/kaggle/input/llm-prompt-recovery/train.csv
/kaggle/input/llm-prompt-recovery/test.csv


2024-03-27 01:04:27.774972: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 01:04:27.775091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 01:04:27.910913: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


end of dependencies


In [2]:
# ---- PROMPT ENGINEERING ----

# Template for Gemma - Engineer to be most effective at generating prompts
template = """Instructions: Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model. You are trying to understand how the original essay was transformed into a new version. Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Only give me the PROMPT. Start directly with the prompt, that's all I need. Output should be only line ONLY.Original Essay:\n{original_text}\n\nRewriten Essay:\n{rewritten_text}\n\nPrompt:\n{rewrite_prompt}"""

In [3]:
# ---- WE NEED BETTER DATA ----

# `3000 Rewritten texts - Prompt recovery Challenge` by @dipamc77
df = pd.read_csv("/kaggle/input/testblendrewrite/rewritten.csv")

# Concat all the rows into a uni-dimensional vector of text data to feed into Gemma
df["prompt"] = df.apply(lambda row: template.format(original_text=row.original_text,
                                                             rewritten_text=row.rewritten_text,
                                                             rewrite_prompt=row.rewrite_prompt), axis=1)
# Isolate our 1D vector into a list
data = df.prompt.tolist()
# Progress log
print("end of preprocessing")

end of preprocessing


In [4]:
'''for x in range(900,905):
    print(df1.loc[(x), 'rewritten_text'])
    print("STOP")'''

'for x in range(900,905):\n    print(df1.loc[(x), \'rewritten_text\'])\n    print("STOP")'

In [5]:
# *Most* of the code after the previous cells will not need to be changed

# Initialize Model - Will need to change based on model in use
model = keras_nlp.models.GemmaCausalLM.from_preset(CFG.preset)
# Optimizer - No need to change
model.backbone.enable_lora(rank=7)
# Input Length - Don't change here, change the CFG class
model.preprocessor.sequence_length = CFG.sequence_length 

# Compile the model with loss, optimizer, and metric
model.compile(
    # Most likely the most optimal loss function
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # ---- Might want to try different optimizers ---- 
    optimizer=keras.optimizers.Adam(learning_rate=3e-5),
    # Not 100% what this does
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
model.fit(data, epochs=CFG.epochs, batch_size=CFG.batch_size)
# Progress Log
print("end of fitting")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


3000/3000 ━━━━━━━━━━━━━━━━━━━━ 2211s 730ms/step - loss: 1.3435 - sparse_categorical_accuracy: 0.6892
end of fitting


In [6]:
# Seed for reproducibility
random.seed(0)

# Take in the 'test' data
test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
test_df['original_text'] = test_df['original_text'].fillna("")
test_df['rewritten_text'] = test_df['rewritten_text'].fillna("")

# Initialize predictions list
prds = []

# Don't need to modify the code of the for loop

# Initialize for loop in a range of length of DataFrame
for num in range(len(test_df)):
    # Save the current row 
    row = test_df.iloc[num]

    # Create the prompt based on the current row
    prompt = template.format(
        original_text=row.original_text,
        rewritten_text=row.rewritten_text,
        rewrite_prompt="",
    )

    # Generate output from model based on prompt
    rewrite_prompt = model.generate(
        prompt, 
        max_length=CFG.max_len,
    ).replace(prompt, "")
    
    # Append the output to predictions list
    prds.append(rewrite_prompt)

# Clean up for submission
test_df.loc[:,'rewrite_prompt'] = prds
df = test_df[['id','rewrite_prompt']]
df['rewrite_prompt'] = df['rewrite_prompt'].fillna("")
df['rewrite_prompt'] = df['rewrite_prompt'].apply(lambda x: 'Improve the essay' if x == "" else x)
# Progress log
print("end of generation")
# Submission
df.to_csv('submission.csv', header=True, index=False)

end of generation


/tmp/ipykernel_26/4214444898.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rewrite_prompt'] = df['rewrite_prompt'].fillna("")
/tmp/ipykernel_26/4214444898.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rewrite_prompt'] = df['rewrite_prompt'].apply(lambda x: 'Improve the essay' if x == "" else x)
